# 1. Data & Library

In [1]:
!pip install bayesian-optimization
import numpy as np
import pandas as pd
import random
import os
import glob
import warnings
warnings.filterwarnings("ignore")

from lightgbm import LGBMRegressor
#from bayes_opt import BayesianOptimization

from tqdm.notebook import tqdm

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=0b35ad4d47b301b0578f4c4e50cc17dbf17533bb32ab8c761cfa2fbf44abe134
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [ ]:
cd 

In [2]:
train = pd.read_csv('/content/drive/MyDrive/DACON/태양광/train.csv')
print(train.shape)

(52560, 9)


In [3]:
test_files = []
import re
numbers = re.compile(r'(\d+)')
def numericalSort(value):
       parts = numbers.split(value)
       parts[1::2] = map(int, parts[1::2])
       return parts

for infile in sorted(glob.glob('/content/drive/MyDrive/DACON/태양광/test/*csv'), key=numericalSort):
  test_files.append(infile)

In [4]:
test_files

['/content/drive/MyDrive/DACON/태양광/test/0.csv',
 '/content/drive/MyDrive/DACON/태양광/test/1.csv',
 '/content/drive/MyDrive/DACON/태양광/test/2.csv',
 '/content/drive/MyDrive/DACON/태양광/test/3.csv',
 '/content/drive/MyDrive/DACON/태양광/test/4.csv',
 '/content/drive/MyDrive/DACON/태양광/test/5.csv',
 '/content/drive/MyDrive/DACON/태양광/test/6.csv',
 '/content/drive/MyDrive/DACON/태양광/test/7.csv',
 '/content/drive/MyDrive/DACON/태양광/test/8.csv',
 '/content/drive/MyDrive/DACON/태양광/test/9.csv',
 '/content/drive/MyDrive/DACON/태양광/test/10.csv',
 '/content/drive/MyDrive/DACON/태양광/test/11.csv',
 '/content/drive/MyDrive/DACON/태양광/test/12.csv',
 '/content/drive/MyDrive/DACON/태양광/test/13.csv',
 '/content/drive/MyDrive/DACON/태양광/test/14.csv',
 '/content/drive/MyDrive/DACON/태양광/test/15.csv',
 '/content/drive/MyDrive/DACON/태양광/test/16.csv',
 '/content/drive/MyDrive/DACON/태양광/test/17.csv',
 '/content/drive/MyDrive/DACON/태양과

In [5]:
test0 = pd.read_csv(test_files[0])
test = []
test = pd.DataFrame(columns=test0.columns)

for i in range(0,len(test_files)):
  test = pd.concat([test, pd.read_csv(test_files[i])])

In [6]:
test.shape

(27216, 9)

# 2. Feature Engineering

### DHI+DNI 변수 생성

In [7]:
train['DHI_DNI']=train['DHI']+train['DNI']
test['DHI_DNI']=test['DHI']+test['DNI']

### 태양의 유무(binary variable) 변수 생성

* TARGET, DHI, DNI 모두 태양의 유무와 직접적인 관계를 지닌다고 판단해서 3개 변수에 대해 생성
* 변수명은 'Sun_{}'으로 통일

In [8]:
# 태양의 유무 변수 생성
train['Sun_TARGET'] = train['TARGET']
train['Sun_DHI'] = train['DHI']
train['Sun_DNI'] = train['DNI']
train['Sun_DHI_DNI'] = train['DHI_DNI']

test['Sun_TARGET'] = test['TARGET']
test['Sun_DHI'] = test['DHI']
test['Sun_DNI'] = test['DNI']
test['Sun_DHI_DNI'] = test['DHI_DNI']


def SUN(x): # 태양의 유무 결정하는 함수
  if x == 0:
    return 0
  else:
    return 1

# apply
train['Sun_TARGET'] = train['Sun_TARGET'].apply(SUN)
train['Sun_DHI'] = train['Sun_DHI'].apply(SUN)
train['Sun_DNI'] = train['Sun_DNI'].apply(SUN)
train['Sun_DHI_DNI'] = train['Sun_DHI_DNI'].apply(SUN)

test['Sun_TARGET'] = test['Sun_TARGET'].apply(SUN)
test['Sun_DHI'] = test['Sun_DHI'].apply(SUN)
test['Sun_DNI'] = test['Sun_DNI'].apply(SUN)
test['Sun_DHI_DNI'] = test['Sun_DHI_DNI'].apply(SUN)

### 해가 떠 있는 시간에 대한 변수 생성

* TARGET, DHI, DNI 모두 해가 떠 있는 시간 직접적인 관계를 지닌다고 판단해서 3개 변수에 대해 생성
* 변수명은 'Sun_{}_hour'로 통일
* 변수명은 hour이지만 실제 단위는 30min

In [10]:
# test셋에 'Sun_{}_hour' 변수를 생성하기 위해 임시로 Day_temp 컬럼 추가
# test는 train과 다르게 day 컬럼이 0일 ~ 6일이 81번 반복되는 상태. 즉, 기존 day 컬럼을 이용해서 변수 생성이 어려움.
test = test.reset_index(drop=True)
day_temp = train['Day'][:567*48]
day_temp = pd.DataFrame(day_temp)
day_temp.columns = ['Day_temp']
test = pd.concat([test, day_temp],axis=1)

train['Sun_TARGET_hour'] = train['Sun_TARGET']
train['Sun_DHI_hour'] = train['Sun_DHI']
train['Sun_DNI_hour'] = train['Sun_DNI']
train['Sun_DHI_DNI_hour'] = train['Sun_DHI_DNI']

for i in range(1095):
  train.loc[train.Day==i,'Sun_TARGET_hour']=train.loc[train.Day==i,'Sun_TARGET'].sum()

for i in range(1095):
  train.loc[train.Day==i,'Sun_DHI_hour']=train.loc[train.Day==i,'Sun_DHI'].sum()

for i in range(1095):
  train.loc[train.Day==i,'Sun_DNI_hour']=train.loc[train.Day==i,'Sun_DNI'].sum()

for i in range(1095):
  train.loc[train.Day==i,'Sun_DHI_DNI_hour']=train.loc[train.Day==i,'Sun_DHI_DNI'].sum()


test['Sun_TARGET_hour'] = test['Sun_TARGET']
test['Sun_DHI_hour'] = test['Sun_DHI']
test['Sun_DNI_hour'] = test['Sun_DNI']
test['Sun_DHI_DNI_hour'] = test['Sun_DHI_DNI']


for i in range(567):
  test.loc[test.Day_temp==i,'Sun_TARGET_hour']=test.loc[test.Day_temp==i,'Sun_TARGET'].sum()

for i in range(567):
  test.loc[test.Day_temp==i,'Sun_DHI_hour']=test.loc[test.Day_temp==i,'Sun_DHI'].sum()

for i in range(567):
  test.loc[test.Day_temp==i,'Sun_DNI_hour']=test.loc[test.Day_temp==i,'Sun_DNI'].sum()

for i in range(567):
  test.loc[test.Day_temp==i,'Sun_DHI_DNI_hour']=test.loc[test.Day_temp==i,'Sun_DHI_DNI'].sum()

In [11]:
train

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,DHI_DNI,Sun_TARGET,Sun_DHI,Sun_DNI,Sun_DHI_DNI,Sun_TARGET_hour,Sun_DHI_hour,Sun_DNI_hour,Sun_DHI_DNI_hour
0,0,0,0,0,0,1.5,69.08,-12,0.0,0,0,0,0,0,18,18,16,18
1,0,0,30,0,0,1.5,69.06,-12,0.0,0,0,0,0,0,18,18,16,18
2,0,1,0,0,0,1.6,71.78,-12,0.0,0,0,0,0,0,18,18,16,18
3,0,1,30,0,0,1.6,71.75,-12,0.0,0,0,0,0,0,18,18,16,18
4,0,2,0,0,0,1.6,75.20,-12,0.0,0,0,0,0,0,18,18,16,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0,0,0,0,0,0,18,18,18,18
52556,1094,22,0,0,0,2.4,66.79,-4,0.0,0,0,0,0,0,18,18,18,18
52557,1094,22,30,0,0,2.2,66.78,-4,0.0,0,0,0,0,0,18,18,18,18
52558,1094,23,0,0,0,2.1,67.72,-4,0.0,0,0,0,0,0,18,18,18,18


### DNI_DHI_plus 변수 생성

* DHI와 DNI이 0이 아닌 행의 개수로 누적으로 해가 떠있는 시간을 구하고 싶었으나
구름 등의 이유(추측일 뿐)로 중간에 0인 값이 존재하여, 두 값들을 더하여 누적 시간을 구하기로함
* TARGET이 0이 아닐때 DHI,DNI 두 값이 동시에 0인 경우는 없음.

* DHI_DNI: 처음부터 두 값을 합쳐서 산출한 값(당일 총합)
* DHI_DNI_plus: DHI와 DNI가 0이 아닌 여부를 따로 구하여 이 여부의 값을 합쳐 만든 변수
 - Sun_DHI가 1이고 Sun_DNI가 1인 경우, Sun_DHI_DNI_plus는 2의 값을 가짐
* {}_accumulate: 누적에 대한 변수들

In [12]:
# 해떠있는 여부의 합
train['Sun_DHI_DNI_plus' ] = train['Sun_DHI']+train['Sun_DNI']
test['Sun_DHI_DNI_plus'] = test['Sun_DHI']+test['Sun_DNI']

# 하루동안 해떠있는 시간의 합
train['Sun_DHI_DNI_plus_hour'] = train.Sun_DHI_hour + train.Sun_DNI_hour
test['Sun_DHI_DNI_plus_hour'] = test.Sun_DHI_hour + test.Sun_DNI_hour

해당 시점이 하루 중 해가 떠있는 몇 시간째인가

In [16]:
# 해당시점이 해가 몇 시간째 떠 있는지 변수 생성 train
for i in tqdm(train.Day.unique()):
  for j in ['TARGET','DNI','DHI','DHI_DNI','DHI_DNI_plus']:
    a=train[train.Day==i]
    for k in a.index:
      if a.loc[k,'Sun_'+str(j)]==0:
        train.loc[k,'Sun_'+str(j)+'_accumulate']=0
      else:
        train.loc[k,'Sun_'+str(j)+'_accumulate']=sum(a.loc[:k,'Sun_'+str(j)])

In [18]:
# 해당시점이 해가 몇 시간째 떠 있는지 변수 생성 test
for i in tqdm(test.Day_temp.unique()):
  for j in ['TARGET','DNI','DHI','DHI_DNI','DHI_DNI_plus']:
    a=test[test.Day_temp==i]
    for k in a.index:
      if a.loc[k,'Sun_'+str(j)]==0:
        test.loc[k,'Sun_'+str(j)+'_accumulate']=0
      else:
        test.loc[k,'Sun_'+str(j)+'_accumulate']=sum(a.loc[:k,'Sun_'+str(j)])

###accumulate를 활용한 변수 생성

* 해당 시점이 하루 중 해가 떠있는 몇 시간째인가/하루 전체 동안 해가 떠있는 시간
*  동일한 시간에(ex. 낮 12시) 해가 떠있다고 해도 겨울과 여름에 따라 그 특정 시간이 해가 뜬지 몇시간째인지는 다를 것이라 생각
*  따라서 이러한 차이를 고려하기 위해 하루 해떠있는 시간을 100이라 했을 떄, 해당 시점이 전체 해뜬 시간의 몇 퍼센트에 위치하는지 파생 변수 생성

In [19]:
for i in tqdm(range(len(train))):
  for j in ['TARGET','DNI','DHI','DHI_DNI','DHI_DNI_plus']:
    if train.loc[i,"Sun_"+str(j)+"_accumulate"]==0:
      train.loc[i,"Sun_"+str(j)+"_per"]=0
    else:
      train.loc[i,"Sun_"+str(j)+"_per"] = (train.loc[i,"Sun_"+str(j)+"_accumulate"]/train.loc[i,"Sun_"+str(j)+"_hour"])*100

In [20]:
for i in tqdm(range(len(test))):
  for j in ['TARGET','DNI','DHI','DHI_DNI','DHI_DNI_plus']:
    if test.loc[i,"Sun_"+str(j)+"_accumulate"]==0:
      test.loc[i,"Sun_"+str(j)+"_per"]=0
    else:
      test.loc[i,"Sun_"+str(j)+"_per"] = (test.loc[i,"Sun_"+str(j)+"_accumulate"]/test.loc[i,"Sun_"+str(j)+"_hour"])*100

In [21]:
# 변수 생성을 위해 만들어놓은 test의 Day_temp 컬럼 삭제
del test['Day_temp']

# 3. 데이터 프레임 재구조화

* 기존에는 2일(TARGET)을 예측하기 위해 전날의 데이터만 사용이 가능
* test 0~80 모두 7일에 대한 데이터(TARGET, 기후 변수들)가 주어진 후 향후 이틀을 예측하는 problem이기 때문에 이를 최대한 활용하는 것이 유리할 것이라 판단
* 하나의 row에 7일 + 2일(TARGET)을 넣기 위해 데이터 재구조화
* 이렇게 하면 더 많은 데이터를 학습에 활용할 수 있게 되어 성능과 generalization의 향상을 기대할 수 있음

### Train 재구조화

In [22]:
train_df=[]
train_df=pd.DataFrame()

for i in tqdm(train.columns[3:]):
  for j in tqdm(range(len(train)-8*48)):
    for n in range(9):
      if n==7 or n==8:
        if i=='TARGET':
          train_df.loc[j,i+'_day'+str(n)]=train.loc[j+48*n,i]
        else:
          pass
      else:
        train_df.loc[j,i+'_day'+str(n)]=train.loc[j+48*n,i]

In [ ]:
# 재구조화 과정에서 시간에 대한 변수를 제외했기 때문에 다시 추가해주는 과정
temp_Hour = train['Hour']*60+train['Minute']
temp_Hour = pd.DataFrame(temp_Hour,columns=['time'])
temp_Hour = temp_Hour.iloc[:len(train_df),:]
train_df = pd.concat([temp_Hour, train_df],axis=1)
train_df.iloc[:48]

### Test 재구조화

In [ ]:
test_df=[]
test_df=pd.DataFrame()

for i in tqdm(test.columns[3:]):
  for b, j in tqdm(enumerate(range(81))):
    a=test.iloc[j*48*7:(j+1)*48*7,:]
    a.index=range(len(a))
    for c,n in enumerate(range(48)):
      for d, m in enumerate(range(7)):
        test_df.loc[b*48+c,i+'_day'+str(m)]=a.loc[n+48*m,i]

In [ ]:
test_temp_Hour = test['Hour']*60+test['Minute']
test_temp_Hour = pd.DataFrame(test_temp_Hour,columns=['time'])
test_temp_Hour = test_temp_Hour.iloc[:len(test_df),:]
test_df = pd.concat([test_temp_Hour, test_df],axis=1)
test_df

In [27]:
train_df.to_csv('/content/drive/MyDrive/DACON/태양광/재구조화/train_재구조화', index=False, encoding='cp949')
test_df.to_csv('/content/drive/MyDrive/DACON/태양광/재구조화/test_재구조화', index=False, encoding='cp949')

In [28]:
train_df

,time,DHI_day0,DHI_day1,DHI_day2,DHI_day3,DHI_day4,DHI_day5,DHI_day6,DNI_day0,DNI_day1,DNI_day2,DNI_day3,DNI_day4,DNI_day5,DNI_day6,WS_day0,WS_day1,WS_day2,WS_day3,WS_day4,WS_day5,WS_day6,RH_day0,RH_day1,RH_day2,RH_day3,RH_day4,RH_day5,RH_day6,T_day0,T_day1,T_day2,T_day3,T_day4,T_day5,T_day6,TARGET_day0,TARGET_day1,TARGET_day2,TARGET_day3,...,Sun_DHI_DNI_plus_accumulate_day2,Sun_DHI_DNI_plus_accumulate_day3,Sun_DHI_DNI_plus_accumulate_day4,Sun_DHI_DNI_plus_accumulate_day5,Sun_DHI_DNI_plus_accumulate_day6,Sun_TARGET_per_day0,Sun_TARGET_per_day1,Sun_TARGET_per_day2,Sun_TARGET_per_day3,Sun_TARGET_per_day4,Sun_TARGET_per_day5,Sun_TARGET_per_day6,Sun_DNI_per_day0,Sun_DNI_per_day1,Sun_DNI_per_day2,Sun_DNI_per_day3,Sun_DNI_per_day4,Sun_DNI_per_day5,Sun_DNI_per_day6,Sun_DHI_per_day0,Sun_DHI_per_day1,Sun_DHI_per_day2,Sun_DHI_per_day3,Sun_DHI_per_day4,Sun_DHI_per_day5,Sun_DHI_per_day6,Sun_DHI_DNI_per_day0,Sun_DHI_DNI_per_day1,Sun_DHI_DNI_per_day2,Sun_DHI_DNI_per_day3,Sun_DHI_DNI_per_day4,Sun_DHI_DNI_per_day5,Sun_DHI_DNI_per_day6,Sun_DHI_DNI_plus_per_day0,Sun_DHI_DNI_plus_per_day1,Sun_DHI_DNI_plus_per_day2,Sun_DHI_DNI_plus_per_day3,Sun_DHI_DNI_plus_per_day4,Sun_DHI_DNI_plus_per_day5,Sun_DHI_DNI_plus_per_day6
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,1.6,2.2,1.8,2.1,1.5,1.9,69.08,90.66,73.80,77.76,72.10,77.41,86.51,-12.0,-10.0,-8.0,-14.0,-5.0,1.0,-2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.5,1.6,2.1,2.0,2.0,1.7,1.8,69.06,90.68,68.20,77.77,72.10,77.42,86.54,-12.0,-10.0,-8.0,-13.0,-5.0,1.0,-2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6,1.6,2.1,2.2,2.0,1.9,1.7,71.78,88.11,69.06,77.69,72.14,76.77,85.72,-12.0,-11.0,-8.0,-13.0,-5.0,1.0,-3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6,1.6,2.1,2.2,2.0,2.0,1.4,71.75,88.11,69.04,71.57,72.11,76.77,85.73,-12.0,-11.0,-8.0,-13.0,-5.0,1.0,-3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,120,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.6,1.6,2.2,2.2,2.0,2.2,1.1,75.20,90.85,69.38,72.62,72.94,76.39,87.04,-12.0,-11.0,-8.0,-13.0,-5.0,1.0,-4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52171,1290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.1,1.6,1.7,1.0,2.7,3.3,3.5,74.26,48.89,79.27,84.85,57.01,61.60,55.97,-7.0,-3.0,-4.0,-8.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52172,1320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.6,1.5,1.7,1.2,2.7,3.3,3.9,70.65,51.96,76.80,82.26,56.82,58.41,54.23,-8.0,-3.0,-4.0,-8.0,-1.0,-1.0,-2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52173,1350,